# [실습] Advanced RAG

RAG의 기본 베이스 체인에서 시작하여, 다양한 기능을 추가해 보겠습니다.

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install --upgrade jsonlines openai langchain langchain-openai langchain-community -q
!pip install chromadb==0.5.3 langchain-chroma tiktoken rank_bm25 -q
!pip install pymupdf pypdf pypdf2 -q

In [33]:
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path("/Users/blueno/UNO/SKALA/SKALA/.env")
load_dotenv(dotenv_path=env_path, override=True)

# 환경 변수 확인
openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [8]:
# 외부에 소스가 있는 경우

#import urllib.request
# urllib.request.urlretrieve(
#     "URL",
#     filename="pdf_doc.pdf"
# )

## 1. Indexing : 데이터 불러오기

./papers 폴더에 포함된 문서들을 pdf 로더로 불러옵니다.

In [6]:
from langchain.schema import Document
from glob import glob

path = './8_papers/*.pdf' #'./drive/MyDrive/8_papers./*.pdf'

glob(path)

['./8_papers/gemma 2.pdf',
 './8_papers/attention.pdf',
 './8_papers/Exaone 3.0.pdf',
 './8_papers/rag.pdf',
 './8_papers/qwen2.pdf',
 './8_papers/phi3.pdf',
 './8_papers/solar.pdf',
 './8_papers/Aya.pdf']

In [7]:
import glob
from langchain_community.document_loaders import PyMuPDFLoader

# 모든 PDF 파일을 glob으로 찾음
pdf_files = glob.glob("./8_papers/*.pdf") #'./drive/MyDrive/8_papers./*.pdf'

# 각 PDF 파일에서 페이지별로 내용을 불러와 하나로 합침
all_papers=[]

for i, path_paper in enumerate(pdf_files):
    loader = PyMuPDFLoader(path_paper)
    pages = loader.load() # PDF는 페이지별로 불러와지므로, 빈 Document에 합치기
    doc = Document(page_content='', metadata = {'index':i, 'source':pages[0].metadata['source']})
    for page in pages:
        doc.page_content += page.page_content
    all_papers.append(doc)

print(len(all_papers))
all_papers[:3]

8


[Document(metadata={'index': 0, 'source': './8_papers/gemma 2.pdf'}, page_content='2024-06-27\nGemma 2: Improving Open Language Models\nat a Practical Size\nGemma Team, Google DeepMind1\nIn this work, we introduce Gemma 2, a new addition to the Gemma family of lightweight, state-of-the-art\nopen models, ranging in scale from 2 billion to 27 billion parameters. In this new version, we apply\nseveral known technical modifications to the Transformer architecture, such as interleaving local-global\nattentions (Beltagy et al., 2020a) and group-query attention (Ainslie et al., 2023). We also train the 2B\nand 9B models with knowledge distillation (Hinton et al., 2015) instead of next token prediction. The\nresulting models deliver the best performance for their size, and even offer competitive alternatives to\nmodels that are 2-3× bigger. We release all our models to the community.\n1. Introduction\nLarge language models (LLMs) have demon-\nstrated strong capabilities in language under-\nsta

In [8]:
import tiktoken

encoder = tiktoken.encoding_for_model('gpt-4o-mini') # token limit 
for paper in all_papers:
    print(len(encoder.encode(paper.page_content)), paper.metadata['source'])

19892 ./8_papers/gemma 2.pdf
10147 ./8_papers/attention.pdf
21014 ./8_papers/Exaone 3.0.pdf
27168 ./8_papers/rag.pdf
25271 ./8_papers/qwen2.pdf
13523 ./8_papers/phi3.pdf
15370 ./8_papers/solar.pdf
25738 ./8_papers/Aya.pdf


# 토큰 단위로 청킹하기

TextSplitter의 `.from_tiktoken_encoder`를 이용합니다. <br>
Token Limit : https://platform.openai.com/settings/organization/limits

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken
token_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini",
    chunk_size=800, # 800 토큰 단위 (GPT-4o-mini 기준) 
    chunk_overlap=80,
)


token_chunks = token_splitter.split_documents(all_papers)
print(len(token_chunks))

230


In [10]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small') # 'text-embedding-3-large'

Chroma().delete_collection()
db = Chroma.from_documents(documents=token_chunks, # 이 코드에서 Chroma.from_document()이 실행되면 embedding을 사용하여 텍스트를 벡터로 변환하고 DB에 저장함. 
                           embedding=embeddings,
                           #persist_directory="./chroma_Web", #  디스크에 저장하고 싶을 때 사용 #'./drive/MyDrive/8_papers./*.pdf'
                           collection_metadata={'hnsw:space':'l2'} # ChromaDB는 고차원 벡터 검색을 빠르게 수행하기 위해 HNSW 알고리즘을 사용, "hnsw:space"는 HNSW 검색 공간 거리 계산 방식 지정 키, "l2"는 L2 거리(유클리드 거리, Euclidean Distance) 의미
                           )
# Top 5 검색하기
retriever = db.as_retriever(search_kwargs={"k": 5}) # db에서 벡터 검색을 수행할 수 있도록 설정함, 데이터 저장이 완료된 이후에 retriever를 통해 검색 가능

# filter 옵션을 통해 특정 메타데이터를 가진 벡터만 검색 가능
# retriever = db.as_retriever(search_kwargs={"k": 5,"filter":{'author':'Sugnryel Lim'}})

retriever.invoke('ai')

[Document(id='d5e4dfbb-c770-46da-bbd6-1f8616c5ba17', metadata={'index': 2, 'source': './8_papers/Exaone 3.0.pdf'}, page_content='the Licensee in connection with the Model that are in addition to, different from, or inconsistent with the terms and\nconditions of this Agreement are not binding on the Licensor and are void.\nBy downloading, installing, or using the EXAONE AI Model, the Licensee acknowledges that it has read,\nunderstood, and agrees to be bound by the terms and conditions of this Agreement.\n20References\n[1] Marah Abdin, Sam Ade Jacobs, Ammar Ahmad Awan, Jyoti Aneja, Ahmed Awadallah, Hany Awadalla, Nguyen\nBach, Amit Bahree, Arash Bakhtiari, Jianmin Bao, Harkirat Behl, Alon Benhaim, Misha Bilenko, Johan Bjorck,\nSébastien Bubeck, Qin Cai, Martin Cai, Caio César Teodoro Mendes, Weizhu Chen, Vishrav Chaudhary, Dong\nChen, Dongdong Chen, Yen-Chun Chen, Yi-Ling Chen, Parul Chopra, Xiyang Dai, Allie Del Giorno, Gustavo\nde Rosa, Matthew Dixon, Ronen Eldan, Victor Fragoso, Dan 

만약 Score를 확인하고 싶다면, Retriever가 아닌 vector store에서 확인이 가능한데요.

In [11]:
# vector store에서 유사도 확인하기
query = "How does Exaone achieve good evaluation results?"
db.similarity_search_with_score(query)

[(Document(id='4383d057-548e-47a0-b486-03385b2d6194', metadata={'index': 2, 'source': './8_papers/Exaone 3.0.pdf'}, page_content='the English general benchmarks in Section 3.1.5, we adopted similar benchmarks KMMLU [35] and KoBEST [21].\nFurthermore, we included Korean subset of Belebele [6] benchmark which is a multiple-choice multilingual machine\nreading comprehension benchmark. The overall results demonstrate that our model outperformed other models on most\nbenchmarks.\nBenchmark\nEXAONE 3.0\n7.8B Inst.\nLlama 3.1\n8B Inst.\nGemma 2\n9B Inst.\nQWEN 2\n7B Inst.\nPhi 3\n7B Inst.\nMistral\n7B Inst.\nKMMLU [35]\n44.5 (2nd)\n41.8\n40.3\n46.5\n37.2\n31.4\nKoBEST-BoolQ [21]\n91.5 (1st)\n87.6\n89.9\n90.2\n76.9\n84.3\nKoBEST-COPA [21]\n85.0 (1st)\n72.8\n60.6\n70.3\n54.5\n62.9\nKoBEST-WiC [21]\n71.2 (1st)\n41.7\n54.3\n65.9\n56.0\n44.6\nKoBEST-HellaSwag [21]\n49.1 (1st)\n44.5\n42.6\n46.8\n34.8\n42.4\nKoBEST-SentiNeg [21]\n98.7 (1st)\n95.2\n72.0\n92.9\n81.0\n84.7\nBelebele [6]\n78.6 (1st)\n73

이를 함수화하여 아래처럼 만들 수도 있습니다.

In [12]:
def retriever_with_score(query):
    docs, scores = zip(*db.similarity_search_with_score(query))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score

    return docs

In [13]:
# Query 검색
# RunnableLambda : 함수를 Runnable로 Wrap

from langchain_core.runnables import RunnableLambda
unique_docs = RunnableLambda(
    retriever_with_score).invoke("How does Exaone achieve good evaluation results?")
# 함수에 직접 invoke를 실행 가능하도록 RunnableLamda()로 묶음
unique_docs

(Document(id='4383d057-548e-47a0-b486-03385b2d6194', metadata={'index': 2, 'source': './8_papers/Exaone 3.0.pdf', 'score': 0.9754084348678589}, page_content='the English general benchmarks in Section 3.1.5, we adopted similar benchmarks KMMLU [35] and KoBEST [21].\nFurthermore, we included Korean subset of Belebele [6] benchmark which is a multiple-choice multilingual machine\nreading comprehension benchmark. The overall results demonstrate that our model outperformed other models on most\nbenchmarks.\nBenchmark\nEXAONE 3.0\n7.8B Inst.\nLlama 3.1\n8B Inst.\nGemma 2\n9B Inst.\nQWEN 2\n7B Inst.\nPhi 3\n7B Inst.\nMistral\n7B Inst.\nKMMLU [35]\n44.5 (2nd)\n41.8\n40.3\n46.5\n37.2\n31.4\nKoBEST-BoolQ [21]\n91.5 (1st)\n87.6\n89.9\n90.2\n76.9\n84.3\nKoBEST-COPA [21]\n85.0 (1st)\n72.8\n60.6\n70.3\n54.5\n62.9\nKoBEST-WiC [21]\n71.2 (1st)\n41.7\n54.3\n65.9\n56.0\n44.6\nKoBEST-HellaSwag [21]\n49.1 (1st)\n44.5\n42.6\n46.8\n34.8\n42.4\nKoBEST-SentiNeg [21]\n98.7 (1st)\n95.2\n72.0\n92.9\n81.0\n84.7\n

RAG의 기본 모델을 구현합니다.

In [14]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)


In [15]:
prompt = ChatPromptTemplate.from_messages([
    ("user", '''당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 한국어로 답변하세요.
정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면, "정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}''')])

prompt.pretty_print()

================================ Human Message =================================

당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 한국어로 답변하세요.
정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면, "정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}


영문 데이터에 맞는 쿼리 생성을 위해, 영문 변환 체인을 구성합니다.

In [16]:
translate_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '주어진 질문을 영어로 변환하세요.'),
        ('user', 'Question: {question}')
    ]
)
translate_chain = translate_prompt | llm | StrOutputParser()

def format_docs(docs):
    return "\n---\n".join([doc.page_content+ '\nURL: '+ doc.metadata['source'] for doc in docs])
    # join : 구분자를 기준으로 스트링 리스트를 하나의 스트링으로 연결

rag_chain = (
    {"context": translate_chain | retriever | format_docs, "question": RunnablePassthrough()}
    # context는 질문을 번역하고 검색한 문서를 텍스트로 변환한 것이며, question은 원래 질문을 그대로 유지
    # retriever : question을 받아서 context 검색: document 반환, format_docs : document 형태를 받아서 텍스트로 변환
    # RunnablePassthrough(): 체인의 입력을 그대로 저장
    | prompt
    # context (검색된 문서)와 question (질문)을 이용하여 LLM에게 입력할 최종 프롬프트를 생성
    | llm
    | StrOutputParser()
)

In [17]:
questions = [
    'Exaone 언어 모델이 다른 모델과 다른 점은 무엇인가요?',
    'Attention이 무엇이며, RNN과 어떻게 다른가요?',
    'Phi-3 언어 모델은 어떤 데이터로 학습했나요?',
    'Solar 언어 모델 구조상의 특이한 점은 무엇인가요?',
    'RAG 특징은 무엇이며, 어떻게 활용하나요?',
    'Qwen 2의 다국어 성능은 어떻게 나타났나요?',
    'Gemma의 스몰 모델은 어떻게 학습했나요?',
    'Aya 모델의 파라미터 수는 각각 몇 개입니까?'
]
result = rag_chain.batch(questions)
for i, ans in enumerate(result):
    ans = ans.replace('.','.\n')
    print(f"Question: {questions[i]}")
    print(f"Answer: {ans}")
    print('---')


Question: Exaone 언어 모델이 다른 모델과 다른 점은 무엇인가요?
Answer: EXAONE 언어 모델은 다음과 같은 점에서 다른 모델들과 차별화됩니다:

1.
 **규모와 성능**: EXAONE 3.
0은 7.
8B 파라미터를 가진 모델로, 특히 한국어에서 뛰어난 성능을 보여줍니다.
 이는 비슷한 크기의 다른 최신 모델들과 비교했을 때 경쟁력 있는 성능을 나타냅니다.


2.
 **이중 언어 지원**: EXAONE 3.
0은 영어와 한국어를 지원하는 이중 언어 모델로, 한국어에 대한 성능이 특히 우수합니다.


3.
 **지침 조정**: EXAONE 3.
0은 지침을 따르는 능력을 향상시키기 위해 두 단계의 후속 훈련(감독 세부 조정 및 직접 선호 최적화)을 수행하여 사용자 상호작용을 보다 잘 반영합니다.


4.
 **데이터 처리 및 준수**: 모델 개발 과정에서 데이터 품질과 법적 문제를 고려하여, 개인 식별 정보(PII)를 포함한 데이터는 제외하고, 법적 위험이 있는 데이터 소스도 배제하는 등 철저한 데이터 준수 절차를 따릅니다.


5.
 **모델 아키텍처**: EXAONE은 디코더 전용 트랜스포머 아키텍처를 기반으로 하며, 최대 4,096 토큰의 컨텍스트 길이를 지원합니다.
 

이러한 특성들은 EXAONE 모델이 전문가 수준의 AI 기능을 일반 대중에게 민주화하는 데 기여하고자 하는 LG AI Research의 비전과 일치합니다.

---
Question: Attention이 무엇이며, RNN과 어떻게 다른가요?
Answer: Attention은 입력 시퀀스의 서로 다른 위치 간의 의존성을 모델링하는 메커니즘으로, 특정 입력의 모든 위치를 고려하여 출력의 각 위치를 생성하는 데 도움을 줍니다.
 이는 입력과 출력 간의 전역적인 의존성을 파악할 수 있게 해주며, 특히 긴 시퀀스에서 중요한 정보를 강조하는 데 유용합니다.


RNN(순환 신경망)은 시퀀스 데이터를 처리하는 전통적인 방법으로, 입력 시퀀스의 각 요소를 순차적으로 처리하여 이

# Multi-Query Retriever   
모호한 쿼리를 검색하는 대신, 다양한 관점에서 Paraphrazing한 쿼리를 사용할 수 있습니다.   
이 때, LLM의 도움을 받을 수 있습니다.

In [18]:
# Multi Query를 확인하기 위한 로깅
import logging

logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [20]:
from langchain.prompts import PromptTemplate

rewrite_prompt = PromptTemplate.from_template(template = """
당신은 AI 언어 모델 어시스턴트입니다.
주어진 사용자 질문을 벡터 데이터베이스에서 관련 문서를 검색하기 위해
3가지 다른 영문 버전으로 생성하는 것이 당신의 작업입니다.
사용자 질문에 대한 여러 관점을 생성함으로써,
당신은 거리 기반 유사성 검색의 한계를 극복할 수 있도록
사용자에게 도움을 주는 것이 목표입니다. 이러한 대체 질문들을
새로운 줄로 구분하여 제공하세요.
---
원본 질문: {question}

""")
from langchain.retrievers.multi_query import MultiQueryRetriever


multi_query_retriever = MultiQueryRetriever.from_llm(
    # MultiQueryRetriever는 LLM을 활용하여 하나의 질문을 여러 개의 검색 쿼리로 변형하는 retriever 
    retriever=db.as_retriever(), # 기존 db(Chroma DB)를 기반으로 검색을 수행하는 retriever
    llm=llm,
    prompt = rewrite_prompt,
    # verbose=True # 검색 과정에 대한 디버깅 정보를 출력할지 여부
)

In [21]:
multi_query_retriever.invoke("Aya는 무슨 모델이에요?")

INFO:langchain.retrievers.multi_query:Generated queries: ['What type of model is Aya?  ', 'Can you explain what kind of model Aya is?  ', 'Could you describe the model that Aya represents?']


[Document(id='3e1990cd-7ec6-4b98-af5b-84d11336af6d', metadata={'index': 7, 'source': './8_papers/Aya.pdf'}, page_content='2stronger models now compared to when mT5 was released, such as the Command R+3, Command\nR4, Llama series [Touvron et al., 2023a;b], Mistral models [Jiang et al., 2023; 2024] and Gemma\nmodels [Gemma-Team, 2024].\nFurthermore, Aya 101 was a 13-billion parameter model designed for breadth, expanding coverage\nto nearly double that achieved by previous models with 101 languages. Due to the well-documented\ncurse of multilinguality [Arivazhagan et al., 2019; Conneau et al., 2019; Pfeiffer et al., 2022], models\nattempting to serve such a broad variety of languages often lag in generative performance on any\ngiven language relative to models dedicated to serving a more focused subset, because of the need\nto share model capacity so widely. For Aya 23, we instead balance breadth and depth, exploring\nthe impact of allocating more capacity to fewer languages (23 language

In [22]:
rag_chain = (
    {"context": multi_query_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [23]:
questions = [
    'Exaone 언어 모델이 다른 모델과 다른 점은 무엇인가요?',
    'Attention이 무엇이며, RNN과 어떻게 다른가요?',
    'Phi-3 언어 모델은 어떤 데이터로 학습했나요?',
    'Solar 언어 모델 구조상의 특이한 점은 무엇인가요?',
    'RAG 특징은 무엇이며, 어떻게 활용하나요?',
    'Qwen 2의 다국어 성능은 어떻게 나타났나요?',
    'Gemma의 스몰 모델은 어떻게 학습했나요?',
    'Aya 모델의 파라미터 수는 각각 몇 개입니까?'
]
result = rag_chain.batch(questions)
for i, ans in enumerate(result):
    ans = ans.replace('.','.\n')
    print(f"Question: {questions[i]}")
    print(f"Answer: {ans}")
    print('---')


INFO:langchain.retrievers.multi_query:Generated queries: ["How did Gemma's small model undergo training?  ", "What training process was used for Gemma's small model?  ", "Can you explain the learning methodology behind Gemma's small model?"]
INFO:langchain.retrievers.multi_query:Generated queries: ['What distinguishes the Exaone language model from other models?  ', 'In what ways is the Exaone language model unique compared to its counterparts?  ', 'How does the Exaone language model differ from other existing models?']
INFO:langchain.retrievers.multi_query:Generated queries: ['What is the number of parameters for each Aya model?  ', 'Can you provide the parameter counts for the different Aya models?  ', 'How many parameters does each version of the Aya model contain?']
INFO:langchain.retrievers.multi_query:Generated queries: ['What is Attention and how does it differ from RNN?  ', 'Can you explain the concept of Attention and its distinctions from RNN?  ', 'What does Attention mean, a

Question: Exaone 언어 모델이 다른 모델과 다른 점은 무엇인가요?
Answer: EXAONE 언어 모델은 다음과 같은 점에서 다른 모델들과 차별화됩니다:

1.
 **한국어 성능**: EXAONE 3.
0은 특히 한국어에서 뛰어난 성능을 보여주며, 유사한 크기의 다른 최신 대형 언어 모델들과 비교했을 때 경쟁력 있는 성능을 발휘합니다.


2.
 **이중 언어 지원**: EXAONE 3.
0은 영어와 한국어를 지원하는 이중 언어 모델로, 한국어의 교착어적 특성을 고려하여 최적화된 토크나이저를 사용합니다.


3.
 **전문 AI 접근성**: EXAONE은 일반 대중이 다양한 분야에서 전문가 수준의 역량을 달성할 수 있도록 돕고, 전문가들이 더 높은 수준의 전문성을 달성할 수 있도록 지원하는 것을 목표로 합니다.


4.
 **개방형 모델**: EXAONE 3.
0 7.
8B 모델은 비상업적 연구 목적으로 공개되어, AI 커뮤니티의 혁신과 협업을 촉진하는 데 기여하고자 합니다.


5.
 **고급 훈련 기법**: EXAONE 3.
0은 감독된 미세 조정(Supervised Fine-Tuning)과 직접 선호 최적화(Direct Preference Optimization)와 같은 고급 후속 훈련 기법을 사용하여 모델의 지시 따르기 능력을 향상시킵니다.


이러한 특징들은 EXAONE 언어 모델이 다른 모델들과 비교했을 때 독특한 장점을 제공하게 합니다.

---
Question: Attention이 무엇이며, RNN과 어떻게 다른가요?
Answer: Attention은 입력 시퀀스의 서로 다른 위치 간의 의존성을 모델링하는 메커니즘으로, 입력과 출력 간의 전역적인 의존성을 계산하는 데 사용됩니다.
 Attention은 입력 시퀀스의 모든 요소가 서로에게 주의를 기울일 수 있도록 하여, 특정 입력 요소가 출력 요소에 미치는 영향을 동적으로 조정할 수 있게 합니다.


RNN(순환 신경망)은 입력 시퀀스의 각 요소를 순차적으로 처리하며, 이전 상태를 기반


### Ensemble Retriever

Lexical 검색인 BM25와 Semantic 검색인 임베딩 방법을 조합할 수도 있습니다.

In [24]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(token_chunks) #, )
# bm25Retriever에 preprocess_func로 별도 정의된 한국어 형태소 분석기를 넣을 수 있음

bm25_retriever.k = 5 # 키워드 기반 검색을 수행 (Lexical 검색), 검색 결과 최대 5개 문서를 반환하게 설정

retriever = db.as_retriever(search_kwargs={"k": 5}) # 벡터 기반 검색을 수행 (Semantic 검색)), 검색 결과 5개 반환

ensemble_retriever = EnsembleRetriever( # 두 검색 방식을 결합하여 가중치를 조정하면서 최적의 검색 결과를 반환
    retrievers=[bm25_retriever, retriever], weights=[0.3, 0.7] # BM25 검색에 30%, 벡터 검색 결고에 70% 가중치 적용
)

In [25]:
rag_chain = (
    {"context": translate_chain | ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [26]:
questions = [
    'Exaone 언어 모델이 다른 모델과 다른 점은 무엇인가요?',
    'Attention이 무엇이며, RNN과 어떻게 다른가요?',
    'Phi-3 언어 모델은 어떤 데이터로 학습했나요?',
    'Solar 언어 모델 구조상의 특이한 점은 무엇인가요?',
    'RAG 특징은 무엇이며, 어떻게 활용하나요?',
    'Qwen 2의 다국어 성능은 어떻게 나타났나요?',
    'Gemma의 스몰 모델은 어떻게 학습했나요?',
    'Aya 모델의 파라미터 수는 각각 몇 개입니까?'
]
result = rag_chain.batch(questions)
for i, ans in enumerate(result):
    ans = ans.replace('.','.\n')
    print(f"Question: {questions[i]}")
    print(f"Answer: {ans}")
    print('---')


Question: Exaone 언어 모델이 다른 모델과 다른 점은 무엇인가요?
Answer: EXAONE 언어 모델은 LG AI Research에서 개발한 첫 번째 오픈 모델로, 7.
8B 파라미터를 가진 instruction-tuned 모델입니다.
 이 모델은 한국어와 영어를 지원하는 이중 언어 모델로, 특히 한국어에서 뛰어난 성능을 보여줍니다.
 EXAONE 3.
0은 다양한 공공 및 내부 벤치마크에서 경쟁력 있는 성능을 입증하였으며, 일반적인 작업과 복잡한 추론에서 우수한 결과를 나타냅니다.
 또한, EXAONE은 전문가 AI를 민주화하는 비전을 가지고 있으며, 일반 대중이 다양한 분야에서 전문가 수준의 역량을 달성할 수 있도록 돕는 것을 목표로 하고 있습니다.

---
Question: Attention이 무엇이며, RNN과 어떻게 다른가요?
Answer: Attention은 입력 데이터의 특정 부분에 집중하여 정보를 처리하는 메커니즘입니다.
 이는 주로 자연어 처리(NLP)와 같은 시퀀스 모델링 작업에서 사용됩니다.
 Attention은 입력 시퀀스의 모든 요소를 동시에 고려할 수 있어, 각 요소 간의 관계를 더 잘 이해할 수 있도록 도와줍니다.
 예를 들어, 문장에서 특정 단어가 다른 단어와 어떻게 연결되는지를 파악하는 데 유용합니다.


반면, RNN(순환 신경망)은 시퀀스 데이터를 처리하기 위해 이전 상태를 기억하고 이를 기반으로 다음 상태를 계산하는 방식으로 작동합니다.
 RNN은 입력 시퀀스를 순차적으로 처리하기 때문에 긴 시퀀스에서는 정보 손실이 발생할 수 있으며, 병렬 처리에 한계가 있습니다.
 

결론적으로, Attention은 입력의 모든 요소를 동시에 고려할 수 있어 장기 의존성을 잘 처리할 수 있는 반면, RNN은 순차적으로 처리하기 때문에 긴 시퀀스에서의 정보 손실과 병렬 처리의 비효율성이 존재합니다.
 Attention 메커니즘은 이러한 RNN의 한계를 극복하기 위해 개발되었습니다.

---
Question: Phi-3 언어 모

# Contextual Retrieval    

Claude가 제안한 Contextual Retrieval은 전체 Context를 활용하여     
청크별 헤더를 추가하는 방법입니다.    

In [28]:
context_prompt = ChatPromptTemplate.from_messages(
    [
        ('user', '''
당신은 문서 분석을 전문으로 하는 AI 어시스턴트입니다. 주어진 문서의 텍스트 한 부분에 대해 간결하고 관련성 있는 문맥을 제공하십시오.

# Input Format

- 문서: `<document> {document} </document>`
- 텍스트 부분: `<chunk> {chunk} </chunk>`

아래의 가이드라인을 참고하여, 이 부분에 대해 간결한 영문 Context을 작성하세요 (2-3문장).

1. 텍스트 부분에서 논의된 주요 주제나 개념을 식별하세요.
2. 문서 전체의 문맥에서 관련 정보나 비교를 언급하세요.
3. 가능한 경우, 이 정보가 문서의 전체적인 주제나 목적과 어떻게 연관되는지를 설명하세요.
4. 중요한 정보를 제공하는 주요 인물, 날짜, 또는 수치를 포함하세요.
5. "이 텍스트는" 또는 "이 섹션에서는 제공한다"와 같은 표현으로 시작하지 말고, 직접적으로 문맥을 서술하세요.

텍스트 부분의 검색 정확성 개선을 위한 문서의 전체 맥락에 위치시키는 간단한 Context만을 출력하세요.
답변은 간결하게 작성하세요.

Context:
        ''')
    ]
)

context_chain = context_prompt | llm | StrOutputParser()


In [29]:
all_papers[1]

Document(metadata={'index': 1, 'source': './8_papers/attention.pdf'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture,

In [30]:
token_chunks[40]

Document(metadata={'index': 1, 'source': './8_papers/attention.pdf'}, page_content='recurrent nets: the difficulty of learning long-term dependencies, 2001.\n[13] Sepp Hochreiter and Jürgen Schmidhuber. Long short-term memory. Neural computation,\n9(8):1735–1780, 1997.\n[14] Zhongqiang Huang and Mary Harper. Self-training PCFG grammars with latent annotations\nacross languages. In Proceedings of the 2009 Conference on Empirical Methods in Natural\nLanguage Processing, pages 832–841. ACL, August 2009.\n[15] Rafal Jozefowicz, Oriol Vinyals, Mike Schuster, Noam Shazeer, and Yonghui Wu. Exploring\nthe limits of language modeling. arXiv preprint arXiv:1602.02410, 2016.\n[16] Łukasz Kaiser and Samy Bengio. Can active memory replace attention? In Advances in Neural\nInformation Processing Systems, (NIPS), 2016.\n[17] Łukasz Kaiser and Ilya Sutskever. Neural GPUs learn algorithms. In International Conference\non Learning Representations (ICLR), 2016.\n[18] Nal Kalchbrenner, Lasse Espeholt, Kar

Context가 잘 생성됐는지 확인해 봅니다.

In [31]:
chunk = token_chunks[40] # 특정 chunk를 선택하고
doc = all_papers[chunk.metadata['index']].page_content # 해당 조각이 속한 원본 문서(all_papers)에서
context = context_chain.invoke({'document':doc, 'chunk':chunk.page_content}) # context_chain을 사용하여 문맥을 생성한 후 출력
print(context)
print('========')
print(chunk.page_content)

The references listed highlight significant contributions to the development of neural network architectures and techniques, particularly in the context of sequence modeling and machine translation. Notably, the work of Hochreiter and Schmidhuber on Long Short-Term Memory (LSTM) networks addresses the challenges of learning long-term dependencies, which is a critical aspect in improving model performance. Additionally, advancements in attention mechanisms, as discussed by various authors, have led to more efficient and effective models, such as the Transformer, which relies entirely on self-attention to enhance translation quality while reducing training time.
recurrent nets: the difficulty of learning long-term dependencies, 2001.
[13] Sepp Hochreiter and Jürgen Schmidhuber. Long short-term memory. Neural computation,
9(8):1735–1780, 1997.
[14] Zhongqiang Huang and Mary Harper. Self-training PCFG grammars with latent annotations
across languages. In Proceedings of the 2009 Conference 

이제 Context 추가 작업을 수행합니다.

In [35]:
from tqdm import tqdm # token_chunks 문맥을 보완하여 업데이트하는 과정을 수행, tqdm을 사용하여 진행 상황을 표시하면서 token_chunks를 순회

for i, chunk in enumerate(tqdm(token_chunks)): # enumerate()를 사용하여 인덱스(i)와 해당 chunk를 함께 가져옴
    doc = all_papers[chunk.metadata['index']].page_content # chunk.metadata['index']: 조각이 속한 원본 문서의 인덱스
    # all_papers[chunk.metadata['index']].page_content: 해당 원본 문서의 전체 내용
    context = context_chain.invoke({'document':doc, 'chunk':chunk.page_content}) # context_chain.invoke()를 사용하여 문맥을 보완
    print('\n'+context)
    print('---')
    token_chunks[i].page_content = context + '\n\n' + token_chunks[i].page_content # 기존 chunk.page_content 앞에 context 추가, 문맥이 추가된 새 page_content로 저장

  0%|          | 1/230 [00:03<12:48,  3.36s/it]


Gemma 2 represents a significant advancement in lightweight open language models, featuring a range of 2 billion to 27 billion parameters and incorporating innovative Transformer architecture modifications such as interleaving local-global attentions and group-query attention. By utilizing knowledge distillation, these models achieve superior performance compared to larger models, demonstrating their effectiveness in language understanding and reasoning tasks. This work underscores the potential of smaller models to compete in various benchmarks, highlighting the importance of training efficiency and architectural innovation in the evolution of large language models.
---


  1%|          | 2/230 [00:06<12:46,  3.36s/it]


Gemma 2 introduces a 27 billion parameter model, significantly enhancing performance through knowledge distillation and advanced architectural techniques like interleaving global and local attention layers and Grouped-Query Attention (GQA). This model competes effectively with larger counterparts across various benchmarks, including question answering and commonsense reasoning, reflecting ongoing efforts to optimize language models for practical applications while ensuring rigorous safety testing and responsible deployment. The advancements in Gemma 2 demonstrate a commitment to pushing the boundaries of open language models, making them accessible for diverse applications.
---


  1%|▏         | 3/230 [00:09<11:09,  2.95s/it]

...


The referenced text discusses significant contributions to the development of large language models, particularly focusing on the Llama series, which includes Llama and Llama 2. These models are part of ongoing research aimed at enhancing multilingual capabilities and addressing biases in language processing. The works cited, including those by Touvron et al. (2023a, 2023b) and Vashishtha et al. (2023), highlight advancements in model architecture and training methodologies that are crucial for improving performance across diverse languages and tasks, aligning with the broader goals of the Aya 23 initiative to enhance multilingual access and equity in AI technologies.
---


100%|█████████▉| 229/230 [25:17<00:07,  7.97s/it]


The provided text includes references to various academic works and papers related to multilingual language models, specifically highlighting the advancements in low-resource language support and the development of the BLOOM+1 model for zero-shot prompting. It also lists the languages supported by the Aya 23 model family, detailing their scripts, linguistic families, and the number of native speakers, emphasizing the model's aim to enhance multilingual capabilities and accessibility in natural language processing. This aligns with the overarching goal of the Aya initiative to broaden the reach of language technologies beyond predominantly English-centric models.
---


100%|██████████| 230/230 [25:26<00:00,  6.64s/it]


The Aya 23 model family supports 23 languages, including widely spoken ones such as Hindi, Italian, Japanese, Korean, and Chinese, along with their respective scripts and language families. This multilingual capability is crucial for enhancing natural language processing technologies, as it aims to bridge the performance gap between English and other languages, thereby promoting equitable access to AI advancements. The document highlights the significant number of native speakers for each language, emphasizing the model's potential impact on a diverse global audience.
---


Chunk의 결과가 추가되었으니, 벡터 데이터베이스를 다시 구성합니다.

In [36]:
Chroma().delete_collection()
db = Chroma.from_documents(documents=token_chunks,
                           embedding=embeddings,
                           collection_metadata={'hnsw:space':'l2'}
                           )

Contextual Header를 이용하기 위해, BM25와 Semantic Search를 결합합니다.

In [38]:
bm25_retriever = BM25Retriever.from_documents(token_chunks)
bm25_retriever.k = 5

retriever = db.as_retriever(search_kwargs={"k": 5})

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
)

In [39]:
questions = [
    'Exaone 언어 모델이 다른 모델과 다른 점은 무엇인가요?',
    'Phi-3 언어 모델은 어떤 데이터로 학습했나요?',
    'Solar 언어 모델 구조상의 특이한 점은 무엇인가요?',
    'Qwen 2의 다국어 성능은 어떻게 나타났나요?',
    'Gemma의 스몰 모델은 어떻게 학습했나요?',
    'Aya 모델의 파라미터 수는 각각 몇 개입니까?'
]
rag_chain = (
    {"context": translate_chain | ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

result = rag_chain.batch(questions)
for i, ans in enumerate(result):
    ans = ans.replace('.','.\n')
    print(f"Question: {questions[i]}")
    print(f"Answer: {ans}")
    print('---')


Question: Exaone 언어 모델이 다른 모델과 다른 점은 무엇인가요?
Answer: EXAONE 언어 모델은 7.
8억 개의 파라미터를 가진 instruction-tuned 모델로, 영어와 한국어 모두에서 뛰어난 성능을 발휘하도록 설계되었습니다.
 이 모델은 디코더 전용 트랜스포머 아키텍처를 기반으로 하며, 최대 4,096 토큰의 컨텍스트 길이를 지원합니다.
 특히, 한국어의 교착어적 특성을 고려하여 이중 언어 토크나이저를 최적화하여 성능과 효율성을 향상시켰습니다.
 

또한, EXAONE은 고품질 데이터 수집 및 법적 준수를 위한 엄격한 프로세스를 통해 개발되었으며, 다양한 공공 및 내부 벤치마크에서 경쟁력 있는 성과를 보여주고 있습니다.
 이 모델은 특히 한국어 작업에서 우수한 성능을 발휘하며, 전문가 수준의 AI 접근성을 민주화하는 LG AI 연구의 목표에 부합합니다.
 

결론적으로, EXAONE은 고급 AI 기능을 제공하고, 다양한 언어 작업에서의 성능을 극대화하기 위해 설계된 점에서 다른 모델들과 차별화됩니다.

---
Question: Phi-3 언어 모델은 어떤 데이터로 학습했나요?
Answer: phi-3-mini 언어 모델은 주로 고도로 필터링된 공개 웹 데이터와 합성 데이터를 결합한 혁신적인 데이터셋을 사용하여 학습되었습니다.
 이 데이터셋은 모델의 일반 지식과 언어 이해를 가르치는 웹 소스와 논리적 추론 및 다양한 전문 기술을 가르치는 합성 데이터를 포함하여 두 개의 단계로 나누어 학습되었습니다.

---
Question: Solar 언어 모델 구조상의 특이한 점은 무엇인가요?
Answer: SOLAR 10.
7B 모델은 "Depth Up-Scaling" (DUS)이라는 새로운 스케일링 방법을 사용하여 구조상의 특이점을 가지고 있습니다.
 이 방법은 전통적인 혼합 전문가 아키텍처에 의존하지 않고, 깊이 기반의 스케일링과 지속적인 사전 훈련을 통해 모델의 성능을 향상시킵니다.
 DUS는 복잡한 변경 없이도 효율적으로 훈련과 추론을 